In [1]:
# Dependencies
import pandas as pd
import json
import numpy as np

pd.set_option('display.max_columns', None)

# TRL - Ranking Algorithm

### This project uses WAR to determine the rankings of specific TRL players. Configuration options for the algorithm can be found in the cell below.

In [2]:
# Configuration Options

# Minimum threshold of games needed for a player in order to receive a rating:
games_threshold = 10

# WAR Calculation multipliers. Must add to 1
offense_multiplier = 0.75
defense_multiplier = 0.2
support_multiplier = 0.05

# Set as False to use classical WAR instead of cWAR
use_cwar = True
# cWAR Coefficient scalar
cwar_scalar = 0.75

# Part 1: Data Wrangling

### In this cell, we can determine which seasons we want to use for WAR computation

In [3]:
# Load all CSV files
Fall2022   = pd.read_csv('csv/per_season/Fall2022.csv', delimiter=';')

Winter2023 = pd.read_csv('csv/per_season/Winter2023.csv', delimiter=';')
Spring2023 = pd.read_csv('csv/per_season/Spring2023.csv', delimiter=';')
Summer2023 = pd.read_csv('csv/per_season/Summer2023.csv', delimiter=';')
Fall2023   = pd.read_csv('csv/per_season/Fall2023.csv', delimiter=';')

Winter2024 = pd.read_csv('csv/per_season/Winter2024.csv', delimiter=';')
Spring2024 = pd.read_csv('csv/per_season/Spring2024.csv', delimiter=';')
Summer2024 = pd.read_csv('csv/per_season/Summer2024.csv', delimiter=';')
Fall2024   = pd.read_csv('csv/per_season/Fall2024.csv', delimiter=';')

Winter2025 = pd.read_csv('csv/per_season/Winter2025.csv', delimiter=';')
Spring2025 = pd.read_csv('csv/per_season/Spring2025.csv', delimiter=';')
Summer2025 = pd.read_csv('csv/per_season/Summer2025.csv', delimiter=';')
Fall2025 = pd.read_csv('csv/per_season/Fall2025.csv', delimiter=';')

all_seasons = [Fall2022, Winter2023, Spring2023, Summer2023, Fall2023, Winter2024, Spring2024, Summer2024, Fall2024, Winter2025, Spring2025, Summer2025, Fall2025]

# Using all season data:
# seasons = all_seasons

# Using only last x seasons:
seasons = [Winter2025, Spring2025, Summer2025, Fall2025]

# Index of first season we want to use (do not modify)
start_season = None
for i, season in enumerate(all_seasons):
    if id(season) == id(seasons[0]):
        start_season = i + 1
        break

### Core merging / data wrangling algorithm. 

#### If additional usernames need to be added for a player, modify the `json/username_mapping.json` file and re-run these cells.

In [4]:
def merge_by_username_per_season(seasons_list):
    """
    Merges and aggregates statistics by player name for each DataFrame in the list separately.
    
    Parameters:
    seasons (list of pd.DataFrame): List of DataFrames to process.
    
    Returns:
    list of pd.DataFrame: List of DataFrames with statistics aggregated by player name for each season.
    """
    
    # Load JSON data from file into a Python dictionary
    with open('json/username_mapping.json', 'r') as file:
        username_mapping = json.load(file)
    
    def merge_and_aggregate(df):
        # Make all player names lowercase
        df['player name'] = df['player name'].str.lower()
        
        # Replace mapped names with their dictionary value
        df['player name'] = df['player name'].map(username_mapping).fillna(df['player name'])
        
        # Choose the statistics we actually want to use. These are total statistics across a season 
        # (i.e. none of these can be averages of any sort)
        columns_to_aggregate = [
            # Core statistics
            'games', 'wins', 'goals', 'assists', 'saves', 'shots', 
            # Other helpful, but not core statistics
            'shots conceded', 'goals conceded', 'amount stolen', 'amount used while supersonic', 
            # Time statistics
            'time supersonic speed', 'time on ground', 'time low in air', 'time high in air',
            'time in front of ball', 'time behind ball', 'time defensive third', 'time neutral third', 'time offensive third', 
            # Demolition statistics
            'demos inflicted', 'demos taken'
        ]
        
        # Merge all the statistics by adding them all up with respect to the player name
        grouped_by_name = df.groupby('player name')[columns_to_aggregate].sum().reset_index()
        
        return grouped_by_name
    
    # Process each season DataFrame
    merged_seasons = [merge_and_aggregate(season_df) for season_df in seasons_list]
    
    return merged_seasons


def calculate_per_game_per_season(seasons_list, threshold):
    """
    Calculate player statistics per game for each DataFrame in the list separately.
    
    Parameters:
    seasons (list of pd.DataFrame): List of DataFrames to process.
    games_threshold (int): Minimum number of games required to be included in the calculations.
    
    Returns:
    list of pd.DataFrame: List of DataFrames with per-game statistics for each season.
    """
    
    def calculate_per_game(season):
        # Calculate player statistics per game
        per_game_stats = season.copy()
        
        # Filter anyone with less than minimum required games
        per_game_stats = per_game_stats[per_game_stats['games'] >= threshold]
        
        # Calculate stats per game
        for col in ['wins', 'goals', 'assists', 'saves', 'shots', 'shots conceded', 'goals conceded', 'amount stolen',
                    'amount used while supersonic', 'time supersonic speed', 'time on ground', 'time low in air',
                    'time high in air', 'time in front of ball', 'time behind ball', 'time defensive third',
                    'time neutral third', 'time offensive third', 'demos inflicted', 'demos taken']:
            per_game_stats[col] = per_game_stats[col] / per_game_stats['games']
        
        # Rename wins to winrate now that it is a percentage:
        per_game_stats = per_game_stats.rename(columns={'wins': 'winrate'})
        
        # Drop unneeded columns
        per_game_stats.drop(columns=['games'], inplace=True)
        per_game_stats.reindex()
        
        return per_game_stats
    
    # Process each season DataFrame
    per_game_seasons = [calculate_per_game(season_df) for season_df in seasons_list]
    
    return per_game_seasons

### Data wrangling complete. 

#### All statistics necessary can be found in the `per_game_stats_by_season` variable, which is a list of dataframes sorted by oldest to most recent season.
#### Each dataframe in the list contains statistics-per-game values for that season, eg. Shots per Game for Spring 2024 

In [5]:
merged_seasons_by_username = merge_by_username_per_season(seasons)
per_game_stats_by_season = calculate_per_game_per_season(merged_seasons_by_username, games_threshold)

# Data wrangling complete. The per_game_statsscaling_fac_by_season is our final array which has every useful statistic in it.
# Display the results:
for i, season_df_per_game in enumerate(per_game_stats_by_season, start=start_season):
    print(f"Season {i} merged statistics:")
    display(season_df_per_game)

Season 10 merged statistics:


,player name,winrate,goals,assists,saves,shots,shots conceded,goals conceded,amount stolen,amount used while supersonic,time supersonic speed,time on ground,time low in air,time high in air,time in front of ball,time behind ball,time defensive third,time neutral third,time offensive third,demos inflicted,demos taken
0,1232char,0.546875,0.765625,0.515625,0.937500,2.140625,7.281250,2.062500,554.109375,347.015625,42.920000,229.052188,126.042031,14.353750,122.839219,246.608125,162.397656,117.814688,89.235000,0.953125,0.890625
1,5.0stormzy,0.575342,0.849315,0.643836,1.342466,3.164384,7.575342,2.397260,599.109589,237.164384,52.345890,199.000822,146.106712,18.121781,109.983699,253.246849,162.466712,120.636164,80.127260,1.232877,0.931507
2,aarav,0.346939,0.755102,0.530612,1.673469,3.102041,8.775510,2.877551,520.346939,395.204082,60.302857,199.414286,138.435714,21.650816,107.380816,252.120000,177.595102,108.936327,72.970816,1.387755,0.857143
3,alex,0.563636,0.981818,0.490909,1.454545,3.527273,7.181818,2.000000,452.981818,260.490909,61.633636,195.184000,156.606545,22.406364,98.621636,275.575636,171.834545,119.214364,83.150909,0.963636,0.818182
4,arby,0.392857,0.732143,0.589286,0.982143,2.535714,7.946429,2.446429,599.482143,260.089286,40.245179,213.766071,131.888929,19.959643,116.073214,249.539107,159.228750,117.526964,88.856607,0.750000,1.053571
5,argon,0.310345,1.000000,0.413793,1.413793,2.862069,10.620690,3.586207,695.413793,440.482759,88.310345,197.099310,167.198276,19.807931,139.094828,245.008966,185.375172,113.825862,84.903103,1.034483,0.758621
6,awe,0.500000,0.640625,0.421875,1.109375,2.359375,7.468750,2.296875,468.187500,243.796875,40.814375,221.315781,128.832969,15.487187,101.490938,264.145625,177.860938,117.860469,69.915156,0.906250,0.828125
7,brictone,0.384615,0.461538,0.358974,1.076923,1.666667,7.538462,2.589744,462.333333,420.487179,40.462821,236.938718,115.146154,10.633846,108.671538,254.046923,174.922821,115.864103,71.933590,0.717949,1.025641
8,capijack,0.567568,0.770270,0.689189,1.486486,2.608108,7.689189,2.175676,420.229730,298.635135,53.096892,211.098108,132.618514,19.409459,94.812297,268.315000,180.320405,115.139324,67.667432,0.608108,1.027027
9,ch,0.563636,0.454545,0.563636,0.727273,2.181818,7.181818,2.000000,607.800000,246.709091,41.312545,217.560000,141.567455,14.778909,117.305273,256.599455,165.098000,119.983818,88.824000,1.872727,0.890909


Season 11 merged statistics:


,player name,winrate,goals,assists,saves,shots,shots conceded,goals conceded,amount stolen,amount used while supersonic,time supersonic speed,time on ground,time low in air,time high in air,time in front of ball,time behind ball,time defensive third,time neutral third,time offensive third,demos inflicted,demos taken
0,bylebukong,0.259259,0.925926,0.296296,1.666667,3.444444,8.555556,3.222222,533.296296,341.925926,59.290741,224.910741,135.135185,20.857407,105.875556,275.026296,190.133333,114.444815,76.322593,0.814815,0.962963
1,capijack,0.382353,0.705882,0.500000,1.647059,2.617647,7.852941,1.941176,489.735294,324.852941,56.275294,214.637647,138.871765,20.793824,90.990000,283.312059,196.726765,108.120294,69.456471,0.764706,1.117647
2,definitelyanna,0.536585,0.780488,0.463415,1.146341,3.000000,7.195122,2.268293,494.170732,304.170732,54.105610,219.710244,140.104390,18.735122,100.465366,278.084390,191.030976,114.985366,72.533659,0.609756,0.975610
3,deso,0.400000,0.825000,0.575000,1.125000,3.175000,7.275000,2.050000,676.900000,463.625000,60.647750,207.596500,142.919250,18.969250,102.857000,266.628750,168.475250,116.565750,84.444750,0.975000,1.250000
4,dies,0.714286,1.257143,0.628571,1.285714,3.371429,7.057143,1.771429,398.228571,136.257143,34.070857,230.185429,125.144000,17.674857,97.969143,275.035143,185.821143,112.535143,74.648286,1.057143,0.942857
6,fernado,0.736842,1.052632,0.789474,1.368421,3.131579,7.184211,1.789474,637.157895,302.552632,55.100789,229.950526,125.641842,20.358158,106.979211,268.972368,175.295263,117.406579,83.247368,1.605263,1.184211
7,front flip freddy,0.588235,0.607843,0.588235,1.156863,2.705882,7.196078,2.254902,416.313725,218.921569,35.445098,233.989412,122.088627,16.073529,99.593922,272.558431,176.056667,118.893137,77.201176,0.882353,1.254902
8,gam,0.444444,1.000000,0.500000,1.472222,2.555556,8.083333,2.250000,619.027778,289.166667,59.149167,215.691667,140.151667,23.147500,106.022778,272.968611,182.651667,117.225833,79.112778,1.194444,0.583333
9,gangster.goose,0.400000,0.350000,0.350000,0.900000,1.350000,7.275000,2.050000,345.625000,198.150000,33.415000,243.560500,116.437000,9.801000,91.774250,278.024250,188.013500,117.132500,64.650500,1.350000,1.250000
10,judin,0.269231,0.269231,0.346154,1.076923,1.576923,8.269231,3.076923,382.000000,205.653846,41.762692,196.747692,145.605000,13.403462,86.926154,268.831923,188.875385,102.113846,64.766923,0.807692,0.692308


Season 12 merged statistics:


,player name,winrate,goals,assists,saves,shots,shots conceded,goals conceded,amount stolen,amount used while supersonic,time supersonic speed,time on ground,time low in air,time high in air,time in front of ball,time behind ball,time defensive third,time neutral third,time offensive third,demos inflicted,demos taken
0,above,0.421053,0.342105,0.394737,0.578947,1.236842,9.263158,2.868421,284.368421,112.894737,19.330263,264.446579,87.909211,3.316053,92.734474,262.937895,180.342632,112.099737,63.230000,0.710526,1.105263
1,arch,0.512821,0.897436,0.512821,0.743590,2.435897,7.589744,2.333333,409.435897,284.410256,37.617436,230.820513,129.101795,15.105128,103.436667,271.589231,174.777179,126.346923,73.900769,0.461538,0.820513
3,ch,0.394737,0.578947,0.368421,1.157895,1.473684,8.052632,2.684211,472.552632,228.578947,41.838421,214.338684,137.592105,13.306842,111.105000,254.132368,179.481053,114.572632,71.184737,1.500000,1.026316
4,definitelyanna,0.590909,0.590909,0.750000,1.068182,2.727273,7.068182,2.295455,510.545455,339.863636,56.941818,204.329318,144.027045,14.959773,95.315455,268.000000,166.913636,117.607045,78.794091,0.818182,1.090909
5,deso,0.577778,1.288889,0.444444,1.022222,3.755556,6.355556,1.955556,591.511111,406.400000,51.706889,206.819333,130.159333,23.949111,96.304889,264.624000,178.670000,105.354889,76.902667,0.933333,1.088889
6,dies,0.300000,0.400000,0.633333,1.633333,2.366667,8.200000,2.800000,414.800000,126.100000,29.878000,229.401333,118.193000,13.189667,98.358333,262.425000,185.092667,109.750000,65.939667,1.200000,0.933333
7,front flip freddy,0.543478,0.826087,0.434783,0.869565,2.695652,7.413043,2.043478,445.326087,278.173913,38.526087,230.961739,121.387391,16.400435,108.374348,260.375652,172.759565,115.276087,80.712826,0.739130,1.152174
8,greensleeves,0.590909,1.681818,0.727273,1.318182,4.454545,7.068182,2.295455,783.113636,259.181818,66.170455,188.222727,155.579091,19.464773,100.521818,262.745909,155.006818,111.196136,97.064091,1.250000,1.113636
9,hotshot,0.512821,1.333333,0.769231,2.179487,3.692308,7.589744,2.333333,668.717949,395.692308,54.776923,200.462821,146.165385,27.835641,89.138462,285.326923,186.343077,113.374359,74.746667,1.000000,0.974359
10,internine,0.300000,0.533333,0.233333,1.066667,1.966667,8.200000,2.800000,546.333333,246.600000,38.977667,218.314667,129.086667,13.381000,115.556000,245.226667,171.274667,108.461667,81.046000,0.600000,0.900000


Season 13 merged statistics:


,player name,winrate,goals,assists,saves,shots,shots conceded,goals conceded,amount stolen,amount used while supersonic,time supersonic speed,time on ground,time low in air,time high in air,time in front of ball,time behind ball,time defensive third,time neutral third,time offensive third,demos inflicted,demos taken
0,above,0.369565,0.152174,0.304348,0.565217,1.065217,9.000000,2.717391,195.782609,137.065217,18.460870,286.928261,90.555000,4.896087,94.799783,287.579348,217.288043,112.256957,52.834783,0.456522,1.195652
1,argon,0.344828,0.689655,0.482759,1.551724,2.586207,8.689655,3.034483,636.586207,384.724138,82.053103,186.577931,156.722069,20.464483,127.418276,236.345862,164.750000,112.534483,86.478966,1.413793,0.862069
2,awe,0.510638,0.829787,0.638298,1.382979,3.042553,7.085106,2.085106,596.191489,349.404255,52.374894,231.637021,123.096383,19.140213,95.612766,278.262766,179.236809,116.198298,78.439362,1.106383,0.872340
3,chasedigi,0.357143,0.261905,0.309524,0.714286,1.285714,7.904762,2.666667,196.547619,106.047619,18.305238,262.745238,93.668333,5.305952,82.831429,278.888571,205.838810,108.737857,47.143571,0.380952,1.142857
4,definitelyanna,0.630435,0.869565,0.543478,1.347826,2.869565,7.086957,1.652174,517.652174,325.391304,59.034130,214.759130,149.044565,17.183043,112.543913,268.442174,175.661522,120.723478,84.602609,0.847826,1.065217
5,deso,0.510638,1.170213,0.446809,1.510638,4.127660,7.085106,2.085106,617.553191,510.042553,62.736809,203.415957,145.947021,23.791915,99.016596,274.138085,175.702340,111.146170,86.305745,0.936170,1.085106
6,dies,0.583333,1.083333,0.645833,1.666667,3.187500,7.541667,2.104167,387.208333,122.541667,29.892083,235.033333,117.438333,13.967083,93.007500,273.431250,186.357500,113.615208,66.467292,1.250000,1.145833
7,front flip freddy,0.543860,0.789474,0.526316,1.508772,2.754386,7.649123,1.947368,460.280702,227.192982,34.418947,223.427544,120.151754,17.933860,101.156491,260.355965,172.262632,111.457719,77.791404,0.666667,0.964912
8,gilan,0.310345,0.344828,0.379310,1.379310,1.862069,8.965517,3.137931,327.620690,260.000000,45.767586,221.917241,130.904828,10.996897,94.821379,268.997241,188.865517,114.202759,60.750690,1.034483,0.896552
9,hotshot,0.392857,1.535714,0.607143,3.214286,4.642857,9.285714,2.785714,505.392857,347.500000,54.273571,204.554643,147.740000,26.923571,83.983929,295.235000,207.487143,105.722500,66.009286,1.107143,1.428571


# Part 2: Load team data

### Teammates and their data is sometimes used during WAR computation, so it is loaded here.

In [6]:
# Helpful methods for loading and locating teammate data

def get_statistic(name, statistics_df, statistic_name):
    # Filter the DataFrame for the given player name
    player_stats = statistics_df[statistics_df['player name'] == name]

    # Check if the player exists in the DataFrame
    if not player_stats.empty:
        # Return the desired statistic value
        return player_stats.iloc[0][statistic_name]
    
    # Return 0 if the player does not exist
    return 0

def get_teammate_stats(teams_one_season, statistics_df, statistic_name):
    # 1. create two new columns with the teammate names
    df_with_teammates = statistics_df.copy()
    df_with_teammates['teammate_1'] = ''
    df_with_teammates['teammate_2'] = ''
    
    for index, row in df_with_teammates.iterrows():
        name = row['player name']
        
        for team_list in teams_one_season:
            if name in team_list:
                teammates_list = team_list.copy()
                teammates_list.remove(name)
                
                if len(teammates_list) >= 2:
                    df_with_teammates.at[index, 'teammate_1'] = teammates_list[0]
                    df_with_teammates.at[index, 'teammate_2'] = teammates_list[1]
                elif len(teammates_list) == 1:
                    df_with_teammates.at[index, 'teammate_1'] = teammates_list[0]
        
    # 2. Load the necessary statistics and add to the dataframe
    df_with_teammates[f'teammate_{statistic_name}'] = 0.0
    for index, row in df_with_teammates.iterrows():
        
        df_with_teammates.at[index, f'teammate_{statistic_name}'] = (
                get_statistic(row['teammate_1'], statistics_df, statistic_name) + get_statistic(row['teammate_2'], statistics_df, statistic_name)
        )
        
        # Add any other necessary teammate statistics here...
    
    # Drop unneeded columns
    df_with_teammates.drop(columns=['teammate_1', 'teammate_2'], inplace=True)
    df_with_teammates.reindex()    
    
    return df_with_teammates

# Load the JSON file containing all the teams
with open('json/teams_per_season.json', 'r') as json_file:
    teams_per_season = json.load(json_file)

# Part 3: WAR Computation

### This is the core algorithm that makes the wheels turn. 
#### Note that the offense, defense, and support multipliers can be modified in the configuration cell, near the top of this notebook.

In [7]:
def calculate_war_per_season(seasons_list, offense_mult=offense_multiplier, defense_mult=defense_multiplier, support_mult=support_multiplier):
    """
    Calculate WAR for each player in each season DataFrame separately.
    
    Parameters:
    seasons (list of pd.DataFrame): List of DataFrames to process. This should be averages per game, sorted by season.
    offense_multiplier (float): Multiplier for offensive statistics.
    defense_multiplier (float): Multiplier for defensive statistics.
    support_multiplier (float): Multiplier for support statistics.
    
    Returns:
    list of pd.DataFrame: List of DataFrames with WAR calculated for each player in each season.
    """
    
    def calculate_war(player_stats_one_season, season_index):
        # Calculate averages for each statistic
        averages_one_season = player_stats_one_season[player_stats_one_season.select_dtypes(include='number').columns].mean()
        
        # Make a new dataframe to store the WAR computations
        rankings_one_season = player_stats_one_season.copy()
        
        # Pull teammate statistics
        teammate_stats_one_season = get_teammate_stats(teams_per_season.get(f"{season_index}"), player_stats_one_season, 'goals')
        
        # Calculate WAR
        rankings_one_season['WAR'] = (
            offense_mult * (
                + (player_stats_one_season['goals'] - averages_one_season['goals'])
                + (player_stats_one_season['assists'] - averages_one_season['assists']) * 0.75
                + (
                        (player_stats_one_season['shots'] - player_stats_one_season['goals'])
                      - (averages_one_season['shots'] - averages_one_season['goals'])
                ) * 0.33          
            ) +
            defense_mult * (
                + (player_stats_one_season['saves'] - averages_one_season['saves']) * 0.6
                - (player_stats_one_season['shots conceded'] - averages_one_season['shots conceded']) * 0.15
                - (player_stats_one_season['goals conceded'] - averages_one_season['goals conceded']) * 0.33
            ) + 
            support_mult * (
                + (player_stats_one_season['demos inflicted'] - averages_one_season['demos inflicted']) * 0.1
                - (player_stats_one_season['demos taken'] - averages_one_season['demos taken']) * 0.1
                + (player_stats_one_season['amount stolen'] - averages_one_season['amount stolen']) * 0.005
            )
        ) + 2
        
        # Store intermediate results
        rankings_one_season.to_csv(f'results/WAR/season_{season_index}.csv', index=False)
        
        # Drop every column except the ones we want to view
        rankings_one_season = rankings_one_season[['player name', 'WAR']]
        
        return rankings_one_season
    
    # Process each season DataFrame
    war_seasons = [calculate_war(season_df, i) for i, season_df in enumerate(seasons_list, start=start_season)]
    
    return war_seasons

In [8]:
# Run the calculate_war_per_season function
war_by_season = calculate_war_per_season(per_game_stats_by_season)

# Display the results
for i, season_df in enumerate(war_by_season, start=start_season):
    print(f"Season {i} WAR statistics:")
    display(season_df.head())

Season 10 WAR statistics:


,player name,WAR
0,1232char,1.899553
1,5.0stormzy,2.297229
2,aarav,2.124241
3,alex,2.381769
4,arby,1.991522


Season 11 WAR statistics:


,player name,WAR
0,bylebukong,2.191482
1,capijack,2.082222
2,definitelyanna,2.132867
3,deso,2.316900
4,dies,2.589288


Season 12 WAR statistics:


,player name,WAR
0,above,1.154243
1,arch,1.933206
3,ch,1.486407
4,definitelyanna,2.067512
5,deso,2.559996


Season 13 WAR statistics:


,player name,WAR
0,above,0.925448
1,argon,1.895751
2,awe,2.245463
3,chasedigi,1.092242
4,definitelyanna,2.171708


# Part 4: Corrected WAR (cWAR)

### This sub-algorithm factors in performance of teammates to award bonuses for "carry" potential.

In [9]:
def calculate_cwar_per_season(seasons_list):
    """
    Calculate cWAR for each player in each season DataFrame separately.
    
    Parameters:
    seasons (list of pd.DataFrame): List of DataFrames to process. This should be WAR per player, sorted by season.
    scaling_factor (float): Multiplier for cWAR computation
    
    Returns:
    list of pd.DataFrame: List of DataFrames with cWAR calculated for each player in each season.
    """
    def calculate_cwar(player_war_one_season, season_index, scaling_factor=cwar_scalar):
        
        # Make a new dataframe which has the sum of WAR of both teammates:
        cwar_one_season = get_teammate_stats(teams_per_season.get(f"{season_index}"), player_war_one_season, 'WAR')
        
        # Drop rows where teammate's WAR sum is zero. This usually happens when a player was a sub i.e. the player
        # had no teammates.
        cwar_one_season = cwar_one_season[cwar_one_season['teammate_WAR'] != 0]
        
        # Compute I_p~t for each player, and put it in a column called 'performance'
        cwar_one_season['performance'] = cwar_one_season['WAR'] / cwar_one_season['teammate_WAR']
        
        # Calculate the average across the 'performance' column
        averages_one_season = cwar_one_season[cwar_one_season.select_dtypes(include='number').columns].mean()
        
        # Add statistic to dataframe, for viewing later - line can be commented out to de-clutter final dataframe
        cwar_one_season['avg_performance'] = averages_one_season['performance']
        
        # Finally, calculate the cWAR of each player
        cwar_one_season['cWAR'] = (
                cwar_one_season['WAR'] 
                + scaling_factor * (
                        (cwar_one_season['performance'] - averages_one_season['performance']) / averages_one_season['performance']
                )
        )
        
        # Store intermediate results
        cwar_one_season.to_csv(f'results/cWAR/season_{season_index}.csv', index=False)
        
        return cwar_one_season
        
    
    # Process each season DataFrame
    cwar_seasons = [calculate_cwar(season_df, i) for i, season_df in enumerate(seasons_list, start=start_season)]
    
    return cwar_seasons

In [10]:
# Run the calculate_cwar_per_season function
cwar_by_season = calculate_cwar_per_season(war_by_season)

# Display the results
for i, season_df in enumerate(cwar_by_season, start=start_season):    
    print(f"Season {i} cWAR:")
    display(season_df.head())

Season 10 cWAR:


,player name,WAR,teammate_WAR,performance,avg_performance,cWAR
0,1232char,1.899553,4.535887,0.418783,0.512959,1.761858
1,5.0stormzy,2.297229,4.214907,0.545025,0.512959,2.344113
2,aarav,2.124241,3.153231,0.673671,0.512959,2.359219
3,alex,2.381769,3.841351,0.620034,0.512959,2.538324
4,arby,1.991522,4.369370,0.455792,0.512959,1.907937


Season 11 cWAR:


,player name,WAR,teammate_WAR,performance,avg_performance,cWAR
0,bylebukong,2.191482,3.027916,0.723759,0.511978,2.501721
1,capijack,2.082222,3.399385,0.612529,0.511978,2.229519
2,definitelyanna,2.132867,3.920814,0.543986,0.511978,2.179755
3,deso,2.316900,3.394394,0.682567,0.511978,2.566796
4,dies,2.589288,4.530753,0.571492,0.511978,2.676469


Season 12 cWAR:


,player name,WAR,teammate_WAR,performance,avg_performance,cWAR
0,above,1.154243,4.620322,0.249819,0.520884,0.763947
1,arch,1.933206,4.237822,0.456179,0.520884,1.840040
3,ch,1.486407,4.051447,0.366883,0.520884,1.264668
4,definitelyanna,2.067512,4.864182,0.425048,0.520884,1.929521
5,deso,2.559996,3.783014,0.676708,0.520884,2.784360


Season 13 cWAR:


,player name,WAR,teammate_WAR,performance,avg_performance,cWAR
0,above,0.925448,4.668535,0.198231,0.579379,0.432055
1,argon,1.895751,3.151662,0.601508,0.579379,1.924398
2,awe,2.245463,3.908521,0.574504,0.579379,2.239153
3,chasedigi,1.092242,4.421200,0.247047,0.579379,0.662042
4,definitelyanna,2.171708,4.046515,0.536686,0.579379,2.116443


# Part 5: WAR Weighting

### More recent seasons will receive a preferential weighting compared to older seasons.
### TODO: This cell currently runs an average across all seasons. Should be modified... at some point..

In [11]:
# Add weights for more recent seasons

# Construct a dictionary to store weighted WAR values for each player
# Dictionary format is - 'player name' : [list, of, WARs]
player_war_dict = {}

all_wars = pd.concat(cwar_by_season)
for _, row in all_wars.iterrows():
    if not row['player name'] in player_war_dict:
        player_war_dict[row['player name']] = []
    if use_cwar:
        player_war_dict[row['player name']].append(row['cWAR'])
    else:
        player_war_dict[row['player name']].append(row['WAR'])

for player_name in player_war_dict: print(f"{player_name} : {player_war_dict[player_name]}")

# Now, weight the WARs such that most recent ratings tewill be favored more
weighted_war_dict = {}
for player_name in player_war_dict:
    weighted_war_dict[player_name] = np.mean(player_war_dict[player_name])

1232char : [1.7618577113966987]
5.0stormzy : [2.344112728843426]
aarav : [2.3592188267594763]
alex : [2.5383237471022544]
arby : [1.9079372824265253]
argon : [2.1901091702122772, 1.9243977518998217]
awe : [1.6801389555912838, 2.2391530600285288]
brictone : [1.2642679175937517]
capijack : [2.120461731596952, 2.2295188083292894]
ch : [1.6162399258930202, 1.2646677855407802]
chyaboi : [2.1538266046459804]
definitelyanna : [1.823062681429675, 2.1797550080414076, 1.92952126445612, 2.116443305945866]
deso : [2.4764925202583035, 2.5667959468086865, 2.7843603290231442, 2.7906937886310157]
dies : [1.9611214818585077, 2.676469166550395, 1.7606832725424624, 2.360337276292725]
fernado : [1.9867834009801995, 2.668277128255059]
front flip freddy : [1.7646239382812963, 1.9296888759131787, 1.912516955123073, 2.0296167079505953]
greensleeves : [3.0524689265920317, 3.566470338839812]
hotshot : [2.857629957627854, 3.329303169936344, 3.8748309482919527]
internine : [1.5756257942001335, 1.303354347149631, 

In [12]:
# Convert dictionary to DataFrame
final_weighted_war = pd.DataFrame.from_dict(weighted_war_dict, orient='index', columns=['WAR'])

# Reset index to make player names a column
final_weighted_war.reset_index(inplace=True)

# Rename the columns
final_weighted_war.rename(columns={'index': 'player name'}, inplace=True)

# Sort by WAR and print final results to a csv
final_weighted_war = final_weighted_war.sort_values(by='WAR', ascending=False)
final_weighted_war.to_csv('results/final_war.csv', index=False)

# Part 6: Team Creation

#### This is using a greedy algorithm to create teams. Essentially, each team tries to make the highest total WAR team that they possibly can. Each team picks the highest rated player out of the remaining players. Then, whichever team has the lowest total WAR gets to pick next.

#### Note that this is not a perfect, be-all-end-all solution as it does not exhaustively test all combinations of teams. However, it does get pretty close.

In [131]:
# If there is anyone we need to manually assign a WAR, we can do it here
# new_players = {
#     'player name': ['ch',      'sqoid', '5.0stormzy', 'peanuthead', 'definitelyanna', 'keaters', 'alex', '1232char', 'psydunk', 'snipeyfriend', 'justin10'],
#     'WAR':         [1.678664,    1.7,     2.284827,      0.8,          2.2,             3.4,       2.8,   1.66,         1.5,          1.6,           1.3  ]
# }

new_players = {
    'player name': [],
    'WAR': []
}

# For players who already exist but we need to overwrite their WAR
final_weighted_war.loc[final_weighted_war['player name'] == 'paz', 'WAR'] = 1.8
final_weighted_war.loc[final_weighted_war['player name'] == 'leagueson', 'WAR'] = 1.77
final_weighted_war.loc[final_weighted_war['player name'] == 'fernado', 'WAR'] = 2.4



final_weighted_war = pd.concat([final_weighted_war, pd.DataFrame(new_players)], ignore_index=True)

# for when we are feeling a lil silly goofy
# final_weighted_war['WAR'] = 0
# final_weighted_war = final_weighted_war.sample(frac=1).reset_index(drop=True)

final_weighted_war = final_weighted_war.sort_values(by='WAR', ascending=False)
final_weighted_war.reset_index(inplace=True, drop=True)
final_weighted_war.to_csv('results/final_war_with_new_players.csv', index=False)

In [132]:
# Signups for Spring 2024. Mostly used as a test.
# playerlist = ['kade', 'mini', 'peak', 'leon', 'snipey', 'greensleeves', 'terminator', 'bylebukong', 'hotshot', 'rubber ducky', 'deso',
#                      'leagueson', 'vpr.vnm', 'tipsy', 'ral days', 'argon', 'fernado', 'pops', 'capi', 'senor brightside', 'arby', 'toucan', 
#                      'tophatbear', 'wika', 'testie', 'waycey', 'king', 'awe', 'phrez', 'front flip freddy', 'lukethighwalkr4', 'dies', 'g_llama',
#                      'renshirokamazaki', 'elatedthug', 'beeholder', 'mistermirz', 'uday', 'gangster.goose']

# Signups for Summer 2024
# playerlist = ['peak', 'hotshot', 'shaunch', 'ral days', 'snipey', 'greencheeze', 'pops', 'vpr.vnm', 'argon', 'deso',
#               'fernado', 'brictone', 'jbassfox', 'king', 'senor brightside', 'tortle', 'capi', 'rutface',
#               'front flip freddy', 'lukethighwalkr4', 'testie', 'tophatbear', 'yopazzy', 'malmoo', 'roo',
#               'theodorelasso', 'renshirokamazaki', 'uday', 'toucan', 'tipsy', 'terminator', 'leon', 'leagueson']

# playerlist = ['mini', 'testie', 'paz', 'keaters', 'senor brightside', 'renshirokamazaki', 'peak', 'front flip freddy', 'judin',
#              'aarav', 'king', 'internine', 'greensleeves', 'capijack', 'lukethighwalkr4', 'hotshot', 'toucan', 'roo',
#              'terminator', 'arby', 'tophatbear', 'leon', 'luma', 'brictone', 'tipsy', 'leagueson', 'waycey', 'alex', 'pops', 'ch',
#              'chyaboi', '5.0stormzy', 'dies', 'deso', 'definitelyanna', 'phrez', 'ral days', 'fernado', '1232char',
#              'rubber ducky', 'vpr.vnm', 'awe', 'argon', 'snipey', 'tortle']

playerlist = ['deso', 'elatedthug', 'testie', 'fernado', 'tortle', 'paz', 'psydunk', 'judin', 'snipey', 'vpr.vnm', 'dies', 'gangster.goose',
             'tophatbear', 'front flip freddy', 'gabo', 'ral days', 'bylebukong', 'leagueson', 'roo', 'lukethighwalkr4', 'toucan',
             'solahz', 'senor brightside', 'yisus', 'definitelyanna', 'capijack', 'renshirokamazaki']#, 'awe', 'waycey', 'rubber ducky']
 

print("Number of signups: ", len(playerlist), "\n")

playerlist = final_weighted_war[final_weighted_war['player name'].isin(playerlist)]

print("Signups w/ data: ", len(playerlist), "\n")

# Sort by WAR
playerlist = playerlist.sort_values(by='WAR', ascending=False)
playerlist.reset_index(inplace=True, drop=True)

print(playerlist, "\n")

playerlist.to_csv('results/playerlist_cur_season.csv', index=False)

# breaks = jpy.jenks_breaks(playerlist['WAR'], n_classes=4)
# playerlist['predicted_tier'] = pd.cut(playerlist['WAR'], bins=breaks, labels=['tier4', 'tier3', 'tier2', 'tier1'], include_lowest=True)

players = playerlist.to_dict('records')


# Initialize teams
teams = [[] for _ in range(len(playerlist) // 3)]
team_wars = [0] * (len(playerlist) // 3)

# List to store full teams
full_teams = []

# Assign players to teams greedily
for player in players:
    if len(teams) == 0:
        break  # If all teams are already full, break the loop
    
    # Find the team with the lowest WAR
    best_team_index = np.argmin(team_wars)
    teams[best_team_index].append(player)
    team_wars[best_team_index] += player['WAR']
    
    # Check if the team is full (3 players)
    if len(teams[best_team_index]) == 3:
        full_teams.append(teams[best_team_index])
        teams.pop(best_team_index)
        team_wars.pop(best_team_index)

# Display the full teams
print("Full Teams:")
for i, team in enumerate(full_teams):
    print(f"Team {i+1}:")
    for player in team:
        print(f"\t{player['player name']} (WAR: {player['WAR']})")
    print(f"\t\tTeam WAR: {sum(player['WAR'] for player in team)}")

# If there are any incomplete teams left, display them as well
if teams:
    print("\nIncomplete Teams:")
    for i, team in enumerate(teams):
        print(f"Incomplete Team {i+1}:")
        for player in team:
            print(f"\t{player['player name']} (WAR: {player['WAR']})")
        print(f"\t\tTeam WAR: {sum(player['WAR'] for player in team)}")

Number of signups:  27 

Signups w/ data:  24 

          player name       WAR
0                deso  2.699745
1            ral days  2.683899
2              solahz  2.568605
3          bylebukong  2.501721
4             fernado  2.400000
5             vpr.vnm  2.209820
6              snipey  2.166143
7                dies  2.132758
8             psydunk  2.043250
9      definitelyanna  1.977446
10           capijack  1.967946
11             toucan  1.904498
12  front flip freddy  1.875710
13             testie  1.833072
14                paz  1.800000
15          leagueson  1.770000
16   senor brightside  1.681051
17         tophatbear  1.429809
18                roo  1.369231
19    lukethighwalkr4  1.360585
20             tortle  1.327796
21              judin  1.240315
22   renshirokamazaki  1.158790
23     gangster.goose  1.114130 

Full Teams:
Team 1:
	snipey (WAR: 2.166143198573145)
	definitelyanna (WAR: 1.9774463179757344)
	senor brightside (WAR: 1.681050878447146)
		Team WAR: 